<a href="https://colab.research.google.com/github/Krishalika/bilateral-vae-collaborative-filtering-recommendation/blob/master/BiVAE_code_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install cornac
# !pip install scrapbook
# !pip install papermill
# !pip install recommenders

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
file_path = '/content/gdrive/My Drive/FYP/BiVAE'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import sys
import os
import torch
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
import gspread as gs

print("System version: {}".format(sys.version))
print("PyTorch version: {}".format(torch.__version__))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]
PyTorch version: 1.13.0+cu116
Cornac version: 1.14.2


In [ ]:
#security-code mapping
mapping_file = pd.read_excel(os.path.join(file_path,"sec_code_mapping.xlsx"))

In [ ]:
#user-stock dataset
data_file = os.path.join(file_path,"user_item_data.csv")
data = pd.read_csv(data_file, names=["userID", "itemID"], skiprows=1)
data.head()

,userID,itemID
0,7,426
1,11,141
2,11,283
3,11,426
4,11,724


In [ ]:
df = data.assign(rating=1)

In [ ]:
train, test = python_random_split(df, 0.75)

In [ ]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 17
Number of items: 18


In [ ]:
# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 50
ENCODER_DIMS = [100]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 300
BATCH_SIZE = 128
LEARNING_RATE = 0.001

In [ ]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/300 [00:00<?, ?it/s]

Took 2.3335 seconds for training.


In [ ]:
top_k = 15
user_input = 56
user = list(train_set.user_ids).index(user_input)

recommended_items = bivae.rank(user)
top_recommendations = list(recommended_items[0])[:top_k]

for item in top_recommendations:
    mask = mapping_file['Code'] == list(train_set.item_ids)[item]
    securitycode = mapping_file.loc[mask, 'SecurityCode']
    print(securitycode.values[0])

# top 15 recommended_items

NIFL.N0000
SHL.N0000
NDB.N0000
DIAL.N0000
GREG.N0000
SEMB.N0000
DFCC.N0000
LFIN.N0000
COMB-BD-27/10/21-C2360-12
CSF.N0000
DFCC-BD-09/11/23-C2367-12.75
VONE.N0000
DIPD.N0000
CCS.N0000
BIL.N0000


In [ ]:
# userID = 0
# itemID = 3
# predicted_score = bivae.score(userID,itemID)
# predicted_score